# Simulation and Noise Models

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../')

We can set up a noise model, following the [Qiskit textbook](https://qiskit.org/textbook/ch-quantum-hardware/error-correction-repetition-code.html#Correcting-errors-in-qubits). We define our noise model to have equal chances of X and Z:

In [3]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

def get_noise(p_err):

    error_gate1 = pauli_error([('X', p_err/2), ('Z', p_err/2), ('I', 1 - p_err)])
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_gate1, "measure") # measurement error is applied to measurements
    noise_model.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"]) # single qubit gate error is applied to x gates
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) # two qubit gate error is applied to cx gates
        
    return noise_model

In [4]:
from qiskit import execute, Aer
from surface_code.circuits import SurfaceCode
from surface_code.fitters import GraphDecoder

We set up a test harness to run through some examples:

In [5]:
def full_loop(code, decoder_simulated, decoder_analytic, noise_model=None):
    # noise_model = get_noise(0.01)
    counts = execute(code.circuit['0'], Aer.get_backend('qasm_simulator'), shots=1).result().get_counts()
    X_errors, Z_errors = code.extract_nodes(code.process_results(counts))

    print("Raw readout:", counts)
    print("Parsed X:", X_errors)
    print("Parsed Z:", Z_errors)
    
    # Analytic Decoder
    flips = {}
    for error_key,errors in dict(zip(("X", "Z"), (X_errors,Z_errors))).items():
        if errors:
            error_graph, paths = decoder_analytic.make_error_graph(errors, error_key)
            matching_graph = decoder_analytic.matching_graph(error_graph,error_key)
            matches = decoder_analytic.matching(matching_graph,error_key)
            flips[error_key] = decoder_analytic.calculate_qubit_flips(matches, paths,error_key)
        else:
            flips[error_key] = {}
    print("----------------")
    print("Decoder results (analytic):")
    print(decoder_analytic.net_qubit_flips(flips["X"], flips["Z"]))

    
    # Simulated Decoder 
    flips = {}
    for error_key,errors in dict(zip(("X", "Z"), (X_errors,Z_errors))).items():
        if errors:
            error_graph = decoder_simulated.make_error_graph(errors, error_key)#when using a simulated syndrome graph, make_error_graph will only return error_graph
            matching_graph = decoder_simulated.matching_graph(error_graph,error_key)
            matches = decoder_simulated.matching(matching_graph,error_key)
            paths = decoder_simulated.analytic_paths(matches,error_key) #simulated syndrome graph does not have nearest neighbor edges that allow for proper error chain determination
            flips[error_key] = decoder_simulated.calculate_qubit_flips(matches, paths,error_key)
        else:
            flips[error_key] = {}
    print("----------------")
    print("Decoder results (simulated):")
    print(decoder_simulated.net_qubit_flips(flips["X"], flips["Z"]))

In [16]:
code = SurfaceCode(3, 3)
decoder_simulated = GraphDecoder(3, 3, simulation=True)
decoder_analytic = GraphDecoder(3, 3)

We can compare `decoder1`, the syndrome graph from simulation, with `decoder2`, the analytic graph:

In [7]:
for i in range(10):
    print("====Run", i, "====")
    full_loop(code, decoder_simulated, decoder_analytic)
    print("=============", "\n")

====Run 0 ====
Raw readout: {'011101101 00100000 00000000': 1}
Parsed X: [(1, 0.5, 1.5)]
Parsed Z: []
----------------
Decoder results (analytic):
{(0.0, 2.0): array([[0, 1],
       [1, 0]])}
----------------
Decoder results (simulated):
{(0.0, 2.0): array([[0, 1],
       [1, 0]])}

====Run 1 ====
Raw readout: {'000000011 10100001 00000000': 1}
Parsed X: [(1, -0.5, 0.5), (1, 0.5, 1.5), (1, 2.5, 1.5)]
Parsed Z: []
----------------
Decoder results (analytic):
{(2.0, 2.0): array([[0, 1],
       [1, 0]]), (0.0, 1.0): array([[0, 1],
       [1, 0]])}
----------------
Decoder results (simulated):
{(2.0, 1.0): array([[0, 1],
       [1, 0]]), (1.0, 1.0): array([[0, 1],
       [1, 0]]), (0.0, 0.0): array([[0, 1],
       [1, 0]])}

====Run 2 ====
Raw readout: {'101011011 00100001 00000000': 1}
Parsed X: [(1, 0.5, 1.5), (1, 2.5, 1.5)]
Parsed Z: []
----------------
Decoder results (analytic):
{(1.0, 1.0): array([[0, 1],
       [1, 0]]), (2.0, 1.0): array([[0, 1],
       [1, 0]])}
----------------
D

And now run with some noise:

In [15]:
for i in range(10):
    print("====Run", i, "====")
    full_loop(code, decoder_simulated, decoder_analytic, noise_model=get_noise(0.1))
    print("=============", "\n")

====Run 0 ====
Raw readout: {'110000011 10000101 00000000': 1}
Parsed X: [(1, -0.5, 0.5), (1, 1.5, 0.5), (1, 2.5, 1.5)]
Parsed Z: []
----------------
Decoder results (analytic):
{(2.0, 1.0): array([[0, 1],
       [1, 0]]), (0.0, 0.0): array([[0, 1],
       [1, 0]])}
----------------
Decoder results (simulated):
{(2.0, 1.0): array([[0, 1],
       [1, 0]]), (0.0, 0.0): array([[0, 1],
       [1, 0]])}

====Run 1 ====
Raw readout: {'110110110 00000001 00000000': 1}
Parsed X: [(1, 2.5, 1.5)]
Parsed Z: []
----------------
Decoder results (analytic):
{(2.0, 2.0): array([[0, 1],
       [1, 0]])}
----------------
Decoder results (simulated):
{(2.0, 2.0): array([[0, 1],
       [1, 0]])}

====Run 2 ====
Raw readout: {'110110101 10100100 00000000': 1}
Parsed X: [(1, -0.5, 0.5), (1, 0.5, 1.5), (1, 1.5, 0.5)]
Parsed Z: []
----------------
Decoder results (analytic):
{(1.0, 1.0): array([[0, 1],
       [1, 0]]), (0.0, 0.0): array([[0, 1],
       [1, 0]])}
----------------
Decoder results (simulated):
